Perform correlation analysis of the dataset to identify relationships that might indicate a causal connection with the bubble issue.

In [2]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

ht_part_number = {
    'type': [0,1],
    'height': [1,3],
    'construction': [3,4],
    'color': [4,5],
    'thickness': [6,7],
    'outside_skin': [7,8],
    'outside_treatment': [8,9],
    'outside_grooving':[9,10],
    'inside_skin': [11,12],
    'inside_grooving': [12,13],
    'length': [14,18],
    'end_stiles': [18,19],
    'backer_stile_placement': [19,20]
}

def read_and_concat_sheets(file_path, selected_columns):
    # Read the Excel file into an ExcelFile object
    excel_file = pd.ExcelFile(file_path)

    # Create an empty list to store the DataFrames
    dfs = []

    # Iterate through each sheet in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the sheet into a DataFrame
        df = excel_file.parse(sheet_name, usecols=selected_columns)        
        # Append the DataFrame to the list
        dfs.append(df)
    # Concatenate all the DataFrames into one
    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df

# data file
file_path = Path('2024_customer_feedback_bubble.xlsx')
selected_columns = ['Line', 'Blend','EndOfPourTimeStamp', 'SmartPartNumber']

# make sure the file exists
if file_path.is_file():
     
    ## convert to csv because it's faster for subsequent processing
     base_name, ext = os.path.splitext(file_path)
     csv_file_path = base_name + '.csv'

     if csv_file_path.is_file():
         print(csv_file_path + ' already exists!')
     else:
         df = read_and_concat_sheets(file_path, selected_columns)
         # Write the DataFrame to a CSV file
         df.to_csv("2024_customer_feedback_bubble.csv", index=False)

print("Loading data . . .")
df = pd.read_csv(csv_file_path)
print("Done")



Done!


In [ ]:
part_dict = df.to_dict()

# disect the part number and make columns for each definition
for option, loc in ht_part_number.items():
    part_dict[option] = {}
    for i, part_no in part_dict['SmartPartNumber'].items():
        part_dict[option][i] = part_no[loc[0]:loc[1]]

# delete the part number and inside grooving
del part_dict['SmartPartNumber']
# del part_dict['inside_grooving']

df = pd.DataFrame.from_dict(part_dict)

# Create a LabelEncoder object
le = LabelEncoder()

# Apply label encoding to all categorical columns
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

# print(df['inside_grooving'].iloc[0:5])

# Perform a correlation analysis using the `corr()` method
correlation_matrix = df.corr()

# Print the correlation matrix
print(correlation_matrix)

The correlation matrix will contain the Pearson correlation coefficients between each pair of variables. A value closer to 1 indicates a strong positive relationship, while values closer to -1 indicate a strong negative relationship. Values close to zero suggest no linear relationship between two variables.

In [ ]:

# Create a heatmap to visualize the correlations
plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.show()


In [ ]:
# Identify the most correlated variables
most_correlated_variables = correlation_matrix.unstack().sort_values(ascending=False)[1:]

# Print the most correlated variables
print('Most correlated variables:')
print(most_correlated_variables)

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


data = {'color': ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'red', 'red']}
df = pd.DataFrame(data)
print(df)  # Print original DataFrame for verification purposes

encoder_color = LabelEncoder()
encoded_colors = encoder_color.fit_transform(df['color'])  # Apply LabelEncoder on the 'color' column and store encoded values in a new variable for demonstration purposes only (optional)
print(encoded_colors)  # Print encoded color values to verify encoding result

    color
0     red
1  orange
2  yellow
3   green
4    blue
5  indigo
6     red
7     red
[4 3 5 1 0 2 4 4]
